In [ ]:
# Kianna's Code

In [ ]:
# Borrowed from Jose
import numpy as np
import pandas as pd
import scipy as sp
import pylab as pl
import scipy.cluster.hierarchy as hier
import scipy.spatial.distance as dist
import matplotlib as plt
%matplotlib inline

In [ ]:
# Borrowed from Jose
gene = pd.read_excel('gene_expression_table4.xlsx')
#print(gene)
#datamatrix=[]
del gene['Unnamed: 9'] # deleted column without any content
#append each column except the first column which is the gene ID
just_expression=gene.iloc[0:373,3:14]
print(just_expression)
just_expression.as_matrix  # transformed dataframe with expression values to array
just_expression.shape
dist_matrix= dist.pdist(just_expression)
dist_square_matrix = dist.squareform(dist_matrix)
#print(dist_square_matrix)

In [ ]:
 # gene

In [ ]:
# Structure of this lasso found on :
# https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/

# Notes on thie lasso function

#    α = 0: Same coefficients as simple linear regression
#    α = ∞: All coefficients zero
#    0 < α < ∞: coefficients between 0 and that of simple linear regression

#    Goal: 10 coefficients that are not 0

from sklearn.linear_model import Lasso
def lasso_regression(data, predictors, alpha, models_to_plot={}):
    #Fit the model
    lassoreg = Lasso(alpha=alpha,normalize=True, max_iter=1e5)
    lassoreg.fit(data[predictors],data['y'])
    y_pred = lassoreg.predict(data[predictors])
    
    #Check if a plot is to be made for the entered alpha
    if alpha in models_to_plot:
        plt.subplot(models_to_plot[alpha])
        plt.tight_layout()
        plt.plot(data['x'],y_pred)
        plt.plot(data['x'],data['y'],'.')
        plt.title('Plot for alpha: %.3g'%alpha)
    
    #Return the result in pre-defined format
    rss = sum((y_pred-data['y'])**2)
    ret = [rss]
    ret.extend([lassoreg.intercept_])
    ret.extend(lassoreg.coef_)
    return ret

In [ ]:
#Initialize predictors to all 15 powers of x
predictors=['x']
predictors.extend(['x_%d'%i for i in range(2,16)])

#Define the alpha values to test
alpha_lasso = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]

#Initialize the dataframe to store coefficients
col = ['rss','intercept'] + ['coef_x_%d'%i for i in range(1,16)]
ind = ['alpha_%.2g'%alpha_lasso[i] for i in range(0,10)]
coef_matrix_lasso = pd.DataFrame(index=ind, columns=col)

#Define the models to plot
models_to_plot = {1e-10:231, 1e-5:232,1e-4:233, 1e-3:234, 1e-2:235, 1:236}

#Iterate over the 10 alpha values:
for i in range(10):
    coef_matrix_lasso.iloc[i,] = lasso_regression(gene, just_expression, alpha_lasso[i], models_to_plot)

In [ ]:
#Coloring the graph
# Compute and plot dendrogram.
fig = pl.figure()
axdendro = fig.add_axes([0.09,0.1,0.2,0.8])
linkageMatrix = hier.linkage(dist_square_matrix, method='centroid') # created linkage matrix
dendrogram = hier.dendrogram(linkageMatrix, orientation='left') # produced dendrogram of the linkage matrix                             #Y = sch.linkage(D, method='centroid')
#Z = sch.dendrogram(Y, orientation='right')
axdendro.set_xticks([])
axdendro.set_yticks([])

# Plot distance matrix.
axmatrix = fig.add_axes([0.3,0.1,0.6,0.8])
index = dendrogram['leaves']
im = axmatrix.matshow(just_expression, aspect='auto', origin='lower')
axmatrix.set_xticks([])
axmatrix.set_yticks([])

# Plot colorbar.
axcolor = fig.add_axes([0.91,0.1,0.02,0.8])
pl.colorbar(im, cax=axcolor, label= 'log2 expression')

# Display and save figure.
fig.show()
fig.savefig('dendrogram.png')
fig.set_figheight(10)


#print(just_expression)
BRCA1_plus= just_expression.iloc[:,0:6]
#print(BRCA1_plus.T)
BRCA1_minus= just_expression.iloc[:,7:12]
#print(BRCA1_minus.T)

BRCA1_plus_means=np.mean(BRCA1_plus.T)
#print(BRCA1_plus_means.shape)
BRCA1_minus_means=np.mean(BRCA1_minus.T)
#print(BRCA1_minus_means.shape)

concatenated_means= np.c_[BRCA1_plus_means,BRCA1_minus_means]
#print(concatenated_means)

expression_change_abs= np.abs(np.diff(concatenated_means))
#print(expression_change_abs.shape)

expression_change_raw= np.diff(concatenated_means)
#print(expression_change_raw)

r=np.append(expression_change_abs,expression_change_raw,1)
print(r)



a=gene.iloc[0:373,0:3]
print(a.shape)
names=['mean','raw mean']
mean_pandas= pd.DataFrame(r,columns=names)
print(mean_pandas.shape)

mean_append= np.append(a,mean_pandas,1)

names2= ['gene name', 'symbol','function','abs value mean difference','raw mean difference']
h=pd.DataFrame(means2_append,columns=names2)

sorted_means=h.sort(['abs value mean difference'],ascending=False)
print(sorted_means)



fig= plt.figure()

#plt.subplot(211)
#plt.plot(np.arange(373),sorted_means.iloc[:,3],':')
fig.set_figheight(10)
fig.set_figwidth(20)
#plt.subplot(212)
#plt.plot(np.arange(373),sorted_means.iloc[:,4],'.')


f=np.unique(sorted_means.iloc[:,2])
print(f)

for i in np.unique(sorted_means.iloc[:,2]):
    plt.plot(np.arange(373),sorted_means.iloc[:,3],':' )






In [ ]:
from ggplot import *

ggplot(sorted_means, aes(y = 'raw mean difference', color = 'function'))